In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
Review_Dataset = df
Review_Dataset.show(10)

In [ ]:
# Create the customers_table DataFrame
customers_table = df.groupby("customer_id").agg({"customer_id":'count'}).withColumnRenamed("count(customer_id)","customer_count")
customers_table.show(20)

In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_table = df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_table.show(10)

In [ ]:
from pyspark.sql.functions import to_date
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent","review_date", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(20)

In [ ]:
# Create the vine_table. DataFrame
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes","vine", "verified_purchase"])
vine_table.show(20)

In [ ]:
D2_df = vine_table.filter("total_votes>=20")
D2_df.show(10)

In [ ]:
from pyspark.sql.functions import col
D2_Percent_Helpful = D2_df.withColumn('percent_helpful_votes',D2_df['helpful_votes']/D2_df['total_votes']).alias('percent_helpful_votes').filter(col("percent_helpful_votes") >= 0.5)
D2_Percent_Helpful.show(10)

In [ ]:
# Filter on exact state
Vine_Review = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "Y").show(10)

In [ ]:
# Filter on exact state
Non_Vine_Review = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "N").show(10)

In [ ]:
Unique_Reviews = D2_Percent_Helpful.drop_duplicates(subset=['review_id']).count()
print(Unique_Reviews)

In [ ]:
Vine_Reviews = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "Y").count()
Vine_Reviews_5Star = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "Y").filter(D2_Percent_Helpful["star_rating"] == "5").count()
Percentage_5_Star = Vine_Reviews_5Star / Vine_Reviews

print(Vine_Reviews)
print(Vine_Reviews_5Star)
print(Percentage_5_Star)

In [ ]:
Non_Vine_Reviews = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "N").count()
Non_Vine_Reviews_5Star = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "N").filter(D2_Percent_Helpful["star_rating"] == "5").count()
Non_Percentage_5_Star = Non_Vine_Reviews_5Star / Non_Vine_Reviews

print(Non_Vine_Reviews)
print(Non_Vine_Reviews_5Star)
print(Non_Percentage_5_Star)